Load a selected ground truth trajectory from test set, 
load the model,
make several predictions with different z's (need to refactor part of agentformer.py still)
Visualize them in one figure.

In [6]:
epochs = [20]
seed = 42
model_id = 'agentformer'


In [7]:
import numpy as np
import os
import sys
import torch

from data.dataloader import data_generator
from utils.torch import *
from utils.config import Config
from model.model_lib import model_dict
from utils.utils import prepare_seed, print_log, mkdir_if_missing
import yaml


def parse_config_yaml(config_file):
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
    return config
# Example usage
config_file = "cfg/eth_ucy/eth/twop_eth_agentformer_pre.yml"
parsed_config = parse_config_yaml(config_file)
from easydict import EasyDict
# Convert the parsed YAML configuration dictionary to an object
cfg = EasyDict(parsed_config)

log = open(os.path.join('./test/visualize', 'log_test.txt'), 'w')

In [8]:
torch.set_default_dtype(torch.float32)
device = torch.device('cuda', index=0) if torch.cuda.is_available() else torch.device('cpu')
if torch.cuda.is_available(): torch.cuda.set_device(0)    
torch.set_grad_enabled(False)

In [9]:
cfg_root_dir = './results/twop_eth_agentformer_pre'
cfg_dir = '%s' % cfg_root_dir
model_dir = '%s/models' % cfg_dir
result_dir = '%s/results' % cfg_dir
log_dir = '%s/log' % cfg_dir
tb_dir = '%s/tb' % cfg_dir
model_path = os.path.join(model_dir, 'model_%04d.p')


In [10]:
prepare_seed(seed)

""" Model """
model = model_dict[model_id](cfg)
model.set_device(device)
model.eval()

""" Load checkpoint """

for epoch in epochs:
    if epoch > 0:
        cp_path = model_path % epoch
        print_log(f'loading model from checkpoint: {cp_path}', log, display=True)
        model_cp = torch.load(cp_path, map_location='cpu')
        model.load_state_dict(model_cp['model_dict'], strict=False)


AgentFormer in  beta  mode.
FutureEncoder in  beta  mode.
FutureDecoder in  beta  mode.
loading model from checkpoint: ./results/twop_eth_agentformer_pre/models/model_0020.p


In [13]:
def get_model_prediction(data, sample_k):
    model.set_data(data)
    recon_motion_3D, _ = model.inference(mode='recon', sample_num=sample_k)
    sample_motion_3D, data = model.inference(mode='infer', sample_num=sample_k, need_weights=False)
    sample_motion_3D = sample_motion_3D.transpose(0, 1).contiguous()
    return recon_motion_3D, sample_motion_3D

def save_prediction(pred, data, suffix, save_dir):
    pred_num = 0
    pred_arr = []
    fut_data, seq_name, frame, valid_id, pred_mask = data['fut_data'], data['seq'], data['frame'], data['valid_id'], data['pred_mask']

    for i in range(len(valid_id)):    # number of agents
        identity = valid_id[i]
        if pred_mask is not None and pred_mask[i] != 1.0:
            continue

        """future frames"""
        for j in range(cfg.future_frames):
            cur_data = fut_data[j]
            if len(cur_data) > 0 and identity in cur_data[:, 1]:
                data = cur_data[cur_data[:, 1] == identity].squeeze()
            else:
                data = most_recent_data.copy()
                data[0] = frame + j + 1
            data[[13, 15]] = pred[i, j].cpu().numpy()   # [13, 15] corresponds to 2D pos
            most_recent_data = data.copy()
            pred_arr.append(data)
        pred_num += 1

    if len(pred_arr) > 0:
        pred_arr = np.vstack(pred_arr)
        indices = [0, 1, 13, 15]            # frame, ID, x, z (remove y which is the height)
        pred_arr = pred_arr[:, indices]
        # save results
        fname = f'{save_dir}/{seq_name}/frame_{int(frame):06d}{suffix}.txt'
        mkdir_if_missing(fname)
        np.savetxt(fname, pred_arr, fmt="%.3f")
    return pred_num

def test_model(generator, save_dir, cfg):
    total_num_pred = 0
    while not generator.is_epoch_end():
        data = generator()
        if data is None:
            continue
        seq_name, frame = data['seq'], data['frame']
        frame = int(frame)
        sys.stdout.write('testing seq: %s, frame: %06d                \r' % (seq_name, frame))  
        sys.stdout.flush()

        gt_motion_3D = torch.stack(data['fut_motion_3D'], dim=0).to(device) * cfg.traj_scale
        with torch.no_grad():
            recon_motion_3D, sample_motion_3D = get_model_prediction(data, cfg.sample_k)
        recon_motion_3D, sample_motion_3D = recon_motion_3D * cfg.traj_scale, sample_motion_3D * cfg.traj_scale

        """save samples"""
        recon_dir = os.path.join(save_dir, 'recon'); mkdir_if_missing(recon_dir)
        sample_dir = os.path.join(save_dir, 'samples'); mkdir_if_missing(sample_dir)
        gt_dir = os.path.join(save_dir, 'gt'); mkdir_if_missing(gt_dir)
        for i in range(sample_motion_3D.shape[0]):
            save_prediction(sample_motion_3D[i], data, f'/sample_{i:03d}', sample_dir)
        save_prediction(recon_motion_3D, data, '', recon_dir)        # save recon
        num_pred = save_prediction(gt_motion_3D, data, '', gt_dir)              # save gt
        total_num_pred += num_pred

    print_log(f'\n\n total_num_pred: {total_num_pred}', log)
    if cfg.dataset == 'nuscenes_pred':
        scene_num = {
            'train': 32186,
            'val': 8560,
            'test': 9041
        }
        assert total_num_pred == scene_num[generator.split]


In [15]:
data_split = ['test']

for split in data_split:
    dataloader = data_generator(cfg, log, split=split, phase='testing')
    save_dir = f'{result_dir}/epoch_{epoch:04d}/{split}'; mkdir_if_missing(save_dir)
    eval_dir = f'{save_dir}/samples'
           
    # for i, data in enumerate(dataloader):
    test_model(dataloader, save_dir, cfg)
    


-------------------------- loading test data --------------------------
loading sequence biwi_eth ...
total num samples: 1142
------------------------------ done --------------------------------

testing seq: biwi_eth, frame: 001226                

 total_num_pred: 364
